In [43]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import sqlite3
from PIL import Image

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.applications import MobileNet
from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import VGG16
from keras.applications import VGG19

import time
import datetime

In [44]:
#Get data
path = 'dataset2'

def getImageWithId(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    
    faces = []
    IDs = []
    
    for imagePath in imagePaths:
        faceImg = Image.open(imagePath).convert('L')
        faceNp = np.array(faceImg, 'uint8')
        Id = int(imagePath.split('.')[1])
        
        faces.append(faceNp)
        IDs.append(Id)
        
        cv2.imshow('training', faceNp)
        cv2.waitKey(10)
    return faces, IDs

In [45]:
#Create train and test
faces, Ids = getImageWithId(path)

x_train = np.array(faces)
y_train = np.array(Ids)

cv2.destroyAllWindows()

In [46]:
x_train = np.reshape(x_train, (x_train.shape[0], 256, 256, 1)) / 255
filenames = os.listdir("dataset2")
categories = []

for f_name in filenames:
    category = f_name.split('.')[1]
    if category == '1':
        categories.append(0)
    elif category == '2':
        categories.append(1)
    elif category == '3':
        categories.append(2)
    elif category == '4':
        categories.append(3)
    elif category == '5':
        categories.append(4)
    '''elif category == '6':
        categories.append(5)
    elif category == '7':
        categories.append(6)
    elif category == '8':
        categories.append(7)
    elif category == '9':
        categories.append(8)'''

print(categories[0])
print(categories[300])

y_train = np_utils.to_categorical(categories, 5)

print(x_train.shape)
print(y_train)
print(y_train.shape)

0
1
(1000, 256, 256, 1)
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
(1000, 5)


In [47]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [48]:
# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [49]:
# Fit the data generator
datagen.fit(x_train)

In [50]:
# Repeat the grayscale image to create 3 channels
x_train = np.repeat(x_train, 3, axis=3)
x_test = np.repeat(x_test, 3, axis=3)

In [37]:
# Load pre-trained VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

# Freeze the pre-trained layers
for layer in vgg.layers:
    layer.trainable = False

# Create model
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Add Dropout layer with dropout rate of 0.5
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
'''# Create MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))'''

'''# Load pre-trained MobileNet model
mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the pre-trained layers
for layer in mobilenet.layers:
    layer.trainable = False

# Create model
model = Sequential()
model.add(mobilenet)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Add Dropout layer with dropout rate of 0.5
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add Dropout layer with dropout rate of 0.5
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


for layer in mobilenet.layers:
    layer.trainable = False

model = Sequential()
model.add(mobilenet)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Add Dropout layer with dropout rate of 0.5
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add Dropout layer with dropout rate of 0.5
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])'''
print('done')

done


In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_3 (Dense)             (None, 1024)              33555456  
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dense_5 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 48,797,509
Trainable params: 34,082,821
Non-trainable params: 14,714,688
_________________________________________________________________


In [38]:
# Train the model with augmented data
start_time = time.time()
history = model.fit(x_train, y_train, batch_size=10, epochs=20, validation_data=(x_test, y_test))
end_time = time.time()
training_time = end_time - start_time

# Đánh giá độ chính xác
_, accuracy = model.evaluate(x_test, y_test)
print("Accuracy:", accuracy)

# Ghi ra file
with open("training_log.txt", "w") as f:
    f.write("Accuracy: {}\n".format(accuracy))
    f.write("Training Time: {} seconds\n".format(training_time))

Epoch 1/20
80/80 [==============================] - 236s 3s/step - loss: 1.2537 - accuracy: 0.8712 - val_loss: 0.2622 - val_accuracy: 0.9650
Epoch 2/20
80/80 [==============================] - 237s 3s/step - loss: 0.0949 - accuracy: 0.9837 - val_loss: 5.5708e-04 - val_accuracy: 1.0000
Epoch 3/20
80/80 [==============================] - 236s 3s/step - loss: 0.0802 - accuracy: 0.9800 - val_loss: 0.0508 - val_accuracy: 0.9850
Epoch 4/20
80/80 [==============================] - 236s 3s/step - loss: 0.0272 - accuracy: 0.9937 - val_loss: 0.0046 - val_accuracy: 0.9950
Epoch 5/20
80/80 [==============================] - 236s 3s/step - loss: 6.4354e-04 - accuracy: 1.0000 - val_loss: 3.6358e-07 - val_accuracy: 1.0000
Epoch 6/20
80/80 [==============================] - 236s 3s/step - loss: 0.0040 - accuracy: 0.9987 - val_loss: 4.0275e-05 - val_accuracy: 1.0000
Epoch 7/20
80/80 [==============================] - 236s 3s/step - loss: 7.1840e-05 - accuracy: 1.0000 - val_loss: 4.9709e-05 - val_accura

In [39]:
model.save('face_transfer_vgg16_5_200.h5')

In [51]:
# Load pre-trained VGG19 model
vgg = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the pre-trained layers
for layer in vgg.layers:
    layer.trainable = False

# Create model
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add Dropout layer with dropout rate of 0.5
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [52]:
# Train the model with augmented data
start_time = time.time()
history = model.fit(x_train, y_train, batch_size=10, epochs=20, validation_data=(x_test, y_test))
end_time = time.time()
training_time = end_time - start_time

# Đánh giá độ chính xác
_, accuracy = model.evaluate(x_test, y_test)
print("Accuracy:", accuracy)

# Ghi ra file
with open("training_log_vgg19.txt", "w") as f:
    f.write("Accuracy: {}\n".format(accuracy))
    f.write("Training Time: {} seconds\n".format(training_time))

Epoch 1/20
80/80 [==============================] - 238s 3s/step - loss: 0.6671 - accuracy: 0.8450 - val_loss: 0.0131 - val_accuracy: 0.9900
Epoch 2/20
80/80 [==============================] - 250s 3s/step - loss: 0.0330 - accuracy: 0.9887 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 3/20
80/80 [==============================] - 258s 3s/step - loss: 0.0593 - accuracy: 0.9800 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 4/20
80/80 [==============================] - 250s 3s/step - loss: 0.0537 - accuracy: 0.9800 - val_loss: 0.0074 - val_accuracy: 0.9950
Epoch 5/20
80/80 [==============================] - 257s 3s/step - loss: 0.0322 - accuracy: 0.9900 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 6/20
80/80 [==============================] - 262s 3s/step - loss: 0.0097 - accuracy: 0.9987 - val_loss: 4.2870e-05 - val_accuracy: 1.0000
Epoch 7/20
80/80 [==============================] - 250s 3s/step - loss: 0.0179 - accuracy: 0.9912 - val_loss: 1.0085e-05 - val_accuracy: 1.0000
Epoch

In [53]:
model.save('face_transfer_vgg19.h5')

In [7]:
from keras.models import load_model
import cv2
import numpy as np
import sqlite3

model = load_model('Face_5_txt_mobilenet.h5')
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

def getProfile(id):
    conn = sqlite3.connect('data.db')
    query = "SELECT * FROM people WHERE ID =" + str(id)
    cursor = conn.execute(query)
    profile = None
    for row in cursor:
        profile = row
    conn.close()
    return profile

cap = cv2.VideoCapture(0)

fontface = cv2.FONT_HERSHEY_SIMPLEX

while True:
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, minSize=(100, 100))
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi_gray = cv2.resize(gray[y: y + h, x: x + w], (256, 256))
        test = np.expand_dims(roi_gray, axis=-1)
        test = np.concatenate((test, test, test), axis=-1)
        test = np.expand_dims(test, axis=0) / 255.0
        t = model.predict(test)
        t = t.reshape(5,)
        print(t[np.argmax(t)])
        id = np.argmax(t) + 1
        profile = getProfile(id)
        if t[np.argmax(t)] > 0.99:
            if profile is not None:
                cv2.putText(frame, str(profile[1]), (x + 10, y + h + 30), fontface, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Unknown", (x + 10, y + h + 30), fontface, 1, (0, 255, 0), 2)
    
    cv2.imshow('image', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 400ms/step
1.0
1/1 [==============================] - 0s 62ms/step
1.0
1/1 [==============================] - 0s 69ms/step
1.0
1/1 [==============================] - 0s 56ms/step
1.0
1/1 [==============================] - 0s 52ms/step
1.0
1/1 [==============================] - 0s 49ms/step
1.0
1/1 [==============================] - 0s 49ms/step
1.0
1/1 [==============================] - 0s 47ms/step
1.0
1/1 [==============================] - 0s 56ms/step
1.0
1/1 [==============================] - 0s 55ms/step
1.0
1/1 [==============================] - 0s 51ms/step
1.0
1/1 [==============================] - 0s 55ms/step
1.0
1/1 [==============================] - 0s 54ms/step
1.0
1/1 [==============================] - 0s 53ms/step
1.0
1/1 [==============================] - 0s 55ms/step
1.0
1/1 [==============================] - 0s 51ms/step
1.0
1/1 [==============================] - 0s 66ms/step
1.0
1/1 [==============================] - 0s 64ms/